<h3>Imports</h3>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random
import sklearn
import tensorflow as tf  
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, GRU, LeakyReLU

<h3>Config Tensorflow to use GPU</h3>

In [2]:
config = tf.compat.v1.ConfigProto()  
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU  
config.log_device_placement = True  # to log device placement (on which device the operation ran)  
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)  
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras  

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080, pci bus id: 0000:08:00.0, compute capability: 7.5



In [30]:
print("progress:%3d%%" % ((1 / 2) * 100), end='\r')

<h3>Create sub-sequences for training</h3>

In [9]:
data_path = "C:\\Users\\Jordan\\Desktop\\MorDNN\\*.txt"

def genDataset(sequence_length):
    global seq_len
    
    global x_train
    global y1_train
    global y2_train

    global num_features
    global output_size_y1
    global output_size_y2
    
    full_sequences = []

    print('Loading raw data...')
    for file in glob.glob(data_path):
        data_file = open(file)
        full_sequences.append( data_file.readlines() )

    num_sequences = len(full_sequences)
        
    seq_len = sequence_length # length of sequences to train on

    data_x = []
    data_y1 = []
    data_y2 = []

    print('Formatting data...')
    for sequence in full_sequences:
        num_samples = len(sequence) // 3

        sequence_x = []
        sequence_y1 = []
        sequence_y2 = []

        # format data into input list and output lists
        for i in range (0, (num_samples - 1) * 3, 3):
            # align current state with outputs for t+1 
            sequence_x.append(np.fromstring(sequence[i], dtype='float', sep=' ')[0:79])
            sequence_y1.append(np.fromstring(sequence[i + 1 + 3], dtype='float', sep=' '))
            sequence_y2.append(np.fromstring(sequence[i + 2 + 3], dtype='float', sep=' '))

        data_x.append(sequence_x)
        data_y1.append(sequence_y1)
        data_y2.append(sequence_y2)

    x_train = []
    y1_train = []
    y2_train = []

    print('Generating training data...')
    for i in range(0, len(data_x)):
        print("progress:%3d%%" % ((i / num_sequences) * 100), end='\r')
        
        num_samples = len(data_x[i])

        j = 0
        while j < num_samples - seq_len:
            subseq_x = []
            subseq_y1 = []
            subseq_y2 = []

            for k in range(j, j + seq_len):
                subseq_x.append(data_x[i][k])
                subseq_y1.append(data_y1[i][k])
                subseq_y2.append(data_y2[i][k])

            x_train.append(subseq_x)
            y1_train.append(subseq_y1)
            y2_train.append(subseq_y2)

            j = j + random.randint(1, 2)
            
    print('Done                       ')

    del full_sequences
    del data_x
    del data_y1
    del data_y2

    x_train = np.array(x_train)
    y1_train = np.array(y1_train)
    y2_train = np.array(y2_train)

    num_features = x_train[0][0].size
    output_size_y1 = y1_train[0][0].size
    output_size_y2 = y2_train[0][0].size

    print(x_train.shape)
    print(y1_train.shape)
    print(y2_train.shape)
    
    
#genDataset(200)
'''
seq_len = 200 # length of subsequences to train on

train_x = []
train_y1 = []
train_y2 = []

# for every full sequence (1 data file)
for i in range(0, len(full_sequences)):
    num_samples = len(full_sequences[i]) // 3
    
    # for each sample in the full sequence, make a subsequence from a window from j to j + seq_len
    # num_samples - 1 b/c we are training on the output at the next timestep
    for j in range(0, (num_samples - 1 - seq_len) * 3, 3):
        subsequence_x = []
        subsequence_y1 = []
        subsequence_y2 = []
        
        # for each sample in the window
        for k in range(j, j + seq_len * 3, 3):
            # game state (input)
            subsequence_x.append( np.fromstring(full_sequences[i][k], dtype='float', sep=' ')[0:79] )

            # outputs (add 3 b/c each sample is 3 lines and we are training on outputs for the timestep of t + 1)
            subsequence_y1.append( np.fromstring(full_sequences[i][k + 1 + 3], dtype='float', sep=' '))
            subsequence_y2.append( np.fromstring(full_sequences[i][k + 2 + 3], dtype='float', sep=' '))
            
        train_x.append(subsequence_x)
        train_y1.append(subsequence_y1)
        train_y2.append(subsequence_y2)
        
train_x = np.array(train_x)
train_y1 = np.array(train_y1)
train_y2 = np.array(train_y2)

print(train_x.shape)
print(train_y1.shape)
print(train_y2.shape)
'''

"\nseq_len = 200 # length of subsequences to train on\n\ntrain_x = []\ntrain_y1 = []\ntrain_y2 = []\n\n# for every full sequence (1 data file)\nfor i in range(0, len(full_sequences)):\n    num_samples = len(full_sequences[i]) // 3\n    \n    # for each sample in the full sequence, make a subsequence from a window from j to j + seq_len\n    # num_samples - 1 b/c we are training on the output at the next timestep\n    for j in range(0, (num_samples - 1 - seq_len) * 3, 3):\n        subsequence_x = []\n        subsequence_y1 = []\n        subsequence_y2 = []\n        \n        # for each sample in the window\n        for k in range(j, j + seq_len * 3, 3):\n            # game state (input)\n            subsequence_x.append( np.fromstring(full_sequences[i][k], dtype='float', sep=' ')[0:79] )\n\n            # outputs (add 3 b/c each sample is 3 lines and we are training on outputs for the timestep of t + 1)\n            subsequence_y1.append( np.fromstring(full_sequences[i][k + 1 + 3], dtype=

<h3>Create Model</h3>

In [7]:
# creates the neural network
# the training model is not stateful and trains on 4 second sequences
# the predcition model is stateful and the internal state will persist between predictions
def createModel(for_training):
    if for_training:
        layer_input = Input(shape = (seq_len, num_features))
        is_stateful = False
    else:
        layer_input = Input(batch_shape = (1, 1, num_features))
        is_stateful = True
        
    '''layer_hid1 = LSTM(400, stateful=is_stateful, return_sequences=True)(layer_input)
    dropout = Dropout(0.2)(layer_hid1)
    layer_hid2 = LSTM(400, stateful=is_stateful, return_sequences=True)(dropout)
    dropout2 = Dropout(0.2)(layer_hid2)
    layer_hid3 = Dense(100)(dropout2)
    layer_hid3 = LeakyReLU(alpha=0.05)(layer_hid3)
    dropout3 = Dropout(0.2)(layer_hid3)'''
    
    layer_hid1 = LSTM(600, stateful=is_stateful, return_sequences=True)(layer_input)
    dropout = Dropout(0.2)(layer_hid1)
    layer_hid2 = LSTM(500, stateful=is_stateful, return_sequences=True)(dropout)
    dropout2 = Dropout(0.2)(layer_hid2)
    layer_hid3 = Dense(200)(dropout2)
    layer_hid3 = LeakyReLU(alpha=0.05)(layer_hid3)
    dropout3 = Dropout(0.2)(layer_hid3)
    
    # button outputs are probabilistic, goal angles are regressional
    layer_out1 = Dense(output_size_y1, activation = 'sigmoid', name='layer_out1')(dropout3)
    layer_out2 = Dense(output_size_y2, activation = 'linear', name='layer_out2')(dropout3)

    model = Model(inputs=layer_input, outputs=[layer_out1, layer_out2])

    if for_training:
        model.compile(
            loss={'layer_out1' : 'binary_crossentropy', 'layer_out2' : 'mean_squared_error'}, 
            loss_weights = {'layer_out1' : 8, 'layer_out2' : 0.5}, # 8, 0.5
            optimizer='adam', 
            metrics=['accuracy']
        )

    #model.summary()
    
    return model

#training_model = createModel(True)

<h3>Load weights</h3>

In [29]:
checkpoint_path_load = "checkpoint_test\\cp.ckpt"
training_model.load_weights(checkpoint_path_load)

In [ ]:
checkpoint_path_load = "checkpoint_test\\cp.ckpt"
checkpoint_path_save = "checkpoint_test\\cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_save,
                                                 save_weights_only=True,
                                                 verbose=1)

for epoch in range(0, 100):
    genDataset(random.randint(100, 400))
    
    training_model = createModel(True)
    training_model.load_weights(checkpoint_path_load)
    
    history = training_model.fit(x_train, {'layer_out1' : y1_train, 'layer_out2' : y2_train},
                    epochs=3, batch_size = 128, shuffle=True,
                    callbacks=[cp_callback])
    
    checkpoint_path_load = checkpoint_path_save
    
    del x_train
    del y1_train
    del y2_train
    

Loading raw data...
Formatting data...
Generating training data...
Done                       
(129301, 111, 79)
(129301, 111, 21)
(129301, 111, 2)
Train on 129301 samples
Epoch 1/3
129280/129301 [============================>.] - ETA: 0s - loss: 1.9750 - layer_out1_loss: 0.0268 - layer_out2_loss: 3.5213 - layer_out1_accuracy: 0.9900 - layer_out2_accuracy: 0.9810
Epoch 00001: saving model to checkpoint_test\cp.ckpt
129301/129301 [==============================] - 87s 672us/sample - loss: 1.9750 - layer_out1_loss: 0.0268 - layer_out2_loss: 3.5215 - layer_out1_accuracy: 0.9900 - layer_out2_accuracy: 0.9810
Epoch 2/3
129280/129301 [============================>.] - ETA: 0s - loss: 1.9783 - layer_out1_loss: 0.0264 - layer_out2_loss: 3.5340 - layer_out1_accuracy: 0.9901 - layer_out2_accuracy: 0.9810
Epoch 00002: saving model to checkpoint_test\cp.ckpt
129301/129301 [==============================] - 82s 636us/sample - loss: 1.9782 - layer_out1_loss: 0.0264 - layer_out2_loss: 3.5330 - layer_

<h3>Train</h3>

In [30]:
checkpoint_path_save = "checkpoint_test\\cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_save,
                                                 save_weights_only=True,
                                                 verbose=1)

history = training_model.fit(x_train, {'layer_out1' : y1_train, 'layer_out2' : y2_train},
                    epochs=500, batch_size = 128, shuffle=True,
                    callbacks=[cp_callback])


Train on 103228 samples
Epoch 1/500
103168/103228 [============================>.] - ETA: 0s - loss: 1.5284 - layer_out1_loss: 0.0166 - layer_out2_loss: 2.7908 - layer_out1_accuracy: 0.9938 - layer_out2_accuracy: 0.9845
Epoch 00001: saving model to checkpoint_test\cp.ckpt
103228/103228 [==============================] - 131s 1ms/sample - loss: 1.5285 - layer_out1_loss: 0.0166 - layer_out2_loss: 2.7910 - layer_out1_accuracy: 0.9938 - layer_out2_accuracy: 0.9845
Epoch 2/500
  2304/103228 [..............................] - ETA: 1:56 - loss: 1.4893 - layer_out1_loss: 0.0164 - layer_out2_loss: 2.7170 - layer_out1_accuracy: 0.9939 - layer_out2_accuracy: 0.9840
Epoch 00002: saving model to checkpoint_test\cp.ckpt
  2304/103228 [..............................] - ETA: 2:01 - loss: 1.4893 - layer_out1_loss: 0.0164 - layer_out2_loss: 2.7170 - layer_out1_accuracy: 0.9939 - layer_out2_accuracy: 0.9840

KeyboardInterrupt: 

<h3>Create and save stateful prediction model</h3>

In [31]:
prediction_model = createModel(False)
prediction_model.load_weights(checkpoint_path_save)
prediction_model.save('mordhai.h5', include_optimizer=False)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(1, 1, 79)]         0                                            
__________________________________________________________________________________________________
lstm_12 (LSTM)                  (1, 1, 600)          1632000     input_7[0][0]                    
__________________________________________________________________________________________________
dropout_18 (Dropout)            (1, 1, 600)          0           lstm_12[0][0]                    
__________________________________________________________________________________________________
lstm_13 (LSTM)                  (1, 1, 500)          2202000     dropout_18[0][0]                 
____________________________________________________________________________________________